In [44]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
#https://medium.com/@14prakash/transfer-learning-using-keras-d804b2e04ef8

from keras import applications
import cv2
import re
import os
import random
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,Conv2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import numpy as np
from keras.utils import to_categorical

Using TensorFlow backend.


In [6]:
img_width=256
img_height=256
oneHot2Name={}





def load_small_img_dataset(path=os.getcwd()):
    data=[]
    labels=[]
    for image in os.listdir(path):
        if image.endswith(('.jpg','.jpeg','.png','bmp')):
            #print(image)
            label=re.findall('^(.*)_.*',image)[0]
            labels.append(label)
            pixels=cv2.imread(os.path.join(path,image))
            pixels=cv2.resize(pixels,(256,256),interpolation=cv2.INTER_CUBIC)
            
            #first make all images of same size using crop
            data.append(pixels)
            
    return data,labels

x,y=load_small_img_dataset('D:\dataset\Image\Face Dataset custom')





persons=set(y)
num_persons=len(set(y))
categorical_mapping={}

#convert into one hot encoding
for i,name in enumerate(persons):
    #print(i,name)
    categorical_mapping[name]=i

output_d=[]

for i_ in y:
    i_=categorical_mapping[i_]
    output_d.append(i_)
output_d=to_categorical(output_d)

from sklearn.utils import shuffle
def test_train_dev_split(input_data,output_data,train=0.7,dev=0.2,test=0.1):
    #make seed for exact results everything
    #random.sort(dataset)
    #np.random.seed(2)
    #np.random.shuffle(input_data)
    #np.random.shuffle(output_data)
    input_data, output_data = shuffle(input_data, output_data, random_state=0)
    split1=int(train*len(input_data))
    split2=int((train+dev)*len(input_data))
    train_input=input_data[:split1]
    dev_input=input_data[split1:split2]
    test_input=input_data[split2:]
    
    
    train_output=output_data[:split1]
    dev_output=output_data[split1:split2]
    test_output=output_data[split2:]
    
    return np.array(train_input),np.array(train_output),np.array(dev_input),np.array(dev_output),np.array(test_input),np.array(test_output)

train_input,train_output,dev_input,dev_output,test_input,test_output=test_train_dev_split(x,output_d)
for i in categorical_mapping.keys():
    oneHot2Name[categorical_mapping[i]]=i




oneHot2Name
cv2.imshow(oneHot2Name[np.argmax(train_output[13])],train_input[13])
cv2.waitKey()
cv2.destroyAllWindows()


# to convert our data type to float32 and normalize our database
train_input=train_input.astype('float32')
dev_input=dev_input.astype('float32')
test_input=test_input.astype('float32')
print(train_input.shape)
print(test_input.shape)


# Z-scoring or Gaussian Normalization
train_input=train_input - np.mean(train_input) / train_input.std()
dev_input=dev_input - np.mean(dev_input) / dev_input.std()
test_input=test_input - np.mean(test_input) / test_input.std()
categorical_mapping


train_input=train_input/255.0
dev_input=dev_input/255.0
test_input=test_input/255.0

(166, 256, 256, 3)
(24, 256, 256, 3)


In [7]:
categorical_mapping

{'Das': 0,
 'Ravi': 1,
 'Surya': 2,
 'Vandana': 3,
 'Faisal': 4,
 'Mummy': 5,
 'Papa': 6,
 'Inder': 7,
 'Nikhil': 8,
 'NikhilAgarwal': 9}

In [8]:
#Saving into numpy arrays
np.save('train_in.npy',train_input)
np.save('train_out.npy',train_output)
np.save('dev_in.npy',dev_input)
np.save('dev_out.npy',dev_output)
np.save('test_in.npy',test_input)
np.save('test_out.npy',test_output)
np.save('num_persons.npy',num_persons)
np.save('categorical_mapping.npy',categorical_mapping)





In [0]:
# Load data from saved numpy arrays


train_input=np.load('train_in.npy')
train_output=np.load('train_out.npy')
dev_input=np.load('dev_in.npy')
dev_output=np.load('dev_out.npy')
test_input=np.load('test_in.npy')
test_output=np.load('test_out.npy')
num_persons=np.load('num_persons.npy')
categorical_mapping=np.load('categorical_mapping.npy')


In [133]:
'''random.seed(20)
random.shuffle(train_input)
random.shuffle(train_output)
random.shuffle(dev_input)
random.shuffle(dev_output)
random.shuffle(test_input)
random.shuffle(test_output)
'''

'random.seed(20)\nrandom.shuffle(train_input)\nrandom.shuffle(train_output)\nrandom.shuffle(dev_input)\nrandom.shuffle(dev_output)\nrandom.shuffle(test_input)\nrandom.shuffle(test_output)\n'

In [134]:
train_output

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0

In [135]:
'''model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
for layer in model.layers[:19]:
    layer.trainable = False'''

'model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))\nfor layer in model.layers[:19]:\n    layer.trainable = False'

In [0]:
#Adding custom Layers 
model=Sequential()
model.add(Conv2D(5,3,input_shape=(100,100,3),activation='relu',padding='same'))


x = model.output
x = Flatten()(x)
x = Dense(12, activation="relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(num_persons, activation="sigmoid")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# compile the model 
model_final.compile( optimizer='sgd',loss = "categorical_crossentropy", metrics=["accuracy"])


In [148]:
model_final.output_shape

(None, array(14, dtype=int32))

In [149]:
train_output

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0

In [10]:
train_output[0:10]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [151]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

'''
# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])
'''

model_final.fit(train_input,train_output,batch_size=10, epochs=1000,validation_data=(dev_input,dev_output),callbacks = [checkpoint, early])


Train on 44 samples, validate on 6 samples
Epoch 1/1000
44/44 [==============================] - 2s 53ms/step - loss: 4.7257 - acc: 0.0909 - val_loss: 9.1752 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to vgg16_1.h5
Epoch 2/1000
44/44 [==============================] - 0s 1ms/step - loss: 6.2787 - acc: 0.0682 - val_loss: 9.1752 - val_acc: 0.0000e+00

Epoch 00002: val_acc did not improve from 0.00000
Epoch 3/1000
44/44 [==============================] - 0s 1ms/step - loss: 5.6377 - acc: 0.0455 - val_loss: 9.1577 - val_acc: 0.0000e+00

Epoch 00003: val_acc did not improve from 0.00000
Epoch 4/1000
44/44 [==============================] - 0s 1ms/step - loss: 6.2237 - acc: 0.0227 - val_loss: 9.1577 - val_acc: 0.0000e+00

Epoch 00004: val_acc did not improve from 0.00000
Epoch 5/1000
44/44 [==============================] - 0s 1ms/step - loss: 4.9894 - acc: 0.0682 - val_loss: 9.1577 - val_acc: 0.0000e+00

Epoch 00005: val_acc did not improve from 

In [152]:
score=model_final.evaluate(test_input,test_output)
#print(score)
print("[INFO] Loss:{0}   Accuracy:{1}".format(score[0],score[1]))


6/6 [==============================] - 0s 2ms/step
[INFO] Loss:9.21034049987793   Accuracy:0.0


In [55]:
categorical_mapping

{'Vishnu': 0,
 'Shreyas': 1,
 'Das': 2,
 'Nikhil': 3,
 'Vandana': 4,
 'Mummy': 5,
 'Ravi': 6,
 'Faisal': 7,
 'Himanshu': 8,
 'vasu': 9,
 'Avinash': 10,
 'Inder': 11,
 'Surya': 12,
 'Joshi': 13}

In [154]:
test_predictions=model_final.predict(test_input)
print(test_predictions)


[[0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.]]


AttributeError: ignored

In [0]:
c=0
for i in test_predictions:
    cv2.imshow(str(oneHot2Name[np.argmax(i)])+str(c)+'.jpg',test_input[c])
    cv2.imwrite('output_'+str(oneHot2Name[np.argmax(i)])+str(c)+'.jpg',test_input[c])
    c+=1
    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [0]:
1/14

0.07142857142857142